In [1]:
import pandas as pd
import jieba

import jieba.posseg as pseg
jieba.set_dictionary('jieba/dict.txt')

jieba.load_userdict('userdict/symptom.txt')
jieba.load_userdict('userdict/time.txt')
jieba.load_userdict('userdict/position.txt')
jieba.load_userdict('userdict/others.txt')
jieba.initialize()

import re
from collections import defaultdict
import matplotlib.pyplot as plt

import time as timer
start_time = timer.time()



kcom = pd.read_csv('Kcom_dr568410605emgcase.csv', lineterminator='\n')
kcom['Kcom'] = kcom['Kcom'].fillna("並無主訴")

print(len(kcom))
kcom.head()

Building prefix dict from /Users/kuanwen/Desktop/自然語言/project/jieba/dict.txt ...
Dumping model to file cache /var/folders/tv/yy9kh1h15dsfg0vcy7vbgtmw0000gn/T/jieba.u936d01f3f8b35df8fcf902039c509f73.cache
Loading model cost 4.102 seconds.
Prefix dict has been built succesfully.


241822


,Pno,Caseno,Kcom
0,15929545,"5,621",今天下午開始雙腳抽筋 現求治
1,15941825,"5,011",剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
2,15965603,"6,001",據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
3,15966387,"6,801",昨天開始頭暈不適 現未改善
4,15968549,"5,441",昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中


In [2]:
p = pd.DataFrame(columns=['Pno','Caseno'])
kcom_out = pd.concat([pd.DataFrame(columns=['Kcom_'+str(i),'Kcom_du'+str(i)]) for i in range(1,11)], axis=1)
kcom_out = pd.concat([p, kcom_out], axis=1)

kcom_out

,Pno,Caseno,Kcom_1,Kcom_du1,Kcom_2,Kcom_du2,Kcom_3,Kcom_du3,Kcom_4,Kcom_du4,...,Kcom_6,Kcom_du6,Kcom_7,Kcom_du7,Kcom_8,Kcom_du8,Kcom_9,Kcom_du9,Kcom_10,Kcom_du10


In [3]:
max_length = 0

with open('output.txt', 'a') as the_file:

    for i in range(len(kcom)):
        split = kcom['Kcom'].loc[i].replace("有","")
        split = split.split()


        t_now = 0
        pre_time = list()
        event_w = list()
        event_now = list()

        for s in split: 
            t_w = list()
            sym_w = list()

            last_t = 0
            last_n = 0
            last_sym = 0
            t = 0
            sym = 0


            line = pseg.cut(s,HMM=False)
            for k,f in line:
                #print('%s %s' % (k,f), end=' ')

                if f == 't':
                    last_n = 0
                    last_sym = 0
                    if t_now == 0:
                        if sym == 1:
                            event = dict()
                            event['time'] = t_w
                            event['sym'] = sym_w
                            event_w.append(event)
                            t_w = list()
                            sym_w = list()
                            sym = 0
                            t_w.append(k)
                            last_t = 1
                            t = 1  
                        elif last_t == 1 and t == 1:
                            t_w.append(k)
                            last_t = 1
                            t = 1
                        elif t == 0:
                            t_w.append(k)
                            last_t = 1
                            t = 1

                elif f == 'tnow' or f == 'tg':
                    last_n = 0
                    last_t = 0
                    last_sym = 0
                    t_now = 1
                    if sym == 1:
                        event = dict()
                        event['time'] = t_w
                        event['sym'] = sym_w
                        event_w.append(event)
                        t_w = list()
                        sym_w = list()
                        sym = 0


                elif f == 'sym':
                    if last_n == 1:
                        sym_w.append(n+k)
                    else:
                        sym_w.append(k)

                    last_n = 0
                    last_t = 0
                    last_sym = 1
                    sym = 1

                elif f == 'f':
                    last_n = 1
                    last_sym = 0
                    n = k
                elif f == 'pos':
                    if last_n == 1:
                        n = n+k
                    else: 
                        n = k
                        
                    last_n = 1
                    last_sym = 0
                elif f == 'm' and last_sym == 1:
                    last_t = 0
                    last_n = 0
                    last_sym = 0
                    if len(t_w) == 0:
                        t_w.append(k)
                        last_t = 1
                
                else:
                    last_t = 0
                    last_n = 0

            if len(sym_w) > 0:
                if t_now == 1:
                    event_now += sym_w
                else:
                    event = dict()
                    if len(t_w) == 0:
                        event['time'] = pre_time
                    else:
                        event['time'] = t_w 
                    event['sym'] = sym_w
                    event_w.append(event)
            elif len(t_w) > 0:
                    pre_time = t_w

        total = list()
        
        time = ''
        for e in event_w:
            event = dict()
            #print(e)
            if len(e['time']) > 0:
                time = ''.join(e['time'])
            event['time'] = time
            event['sym'] = e['sym']

            total.append(event)
        
        
        for idx, e in reversed(list(enumerate(total))):
            if idx != 0:
                if e['time'] == total[idx-1]['time']:
                    #print(e)
                    total[idx-1]['sym'] += e['sym']
                    del total[idx]
        
        
    
        
        if len(event_now) > 0:
            total.append({'time': '現在', 'sym': event_now})
            #for e_now in event_now:
                #total.append({'time': '現在', 'sym': [e_now]})

        '''
        if len(total) > max_length:
            max_length = len(total)
        '''
        
        
        '''
        p = pd.DataFrame(columns=['Pno','Caseno'])
        temp = pd.concat([pd.DataFrame(columns=['Kcom_'+str(t),'Kcom_du'+str(t)]) for t in range(1,11)], axis=1)
        temp = pd.concat([p, temp], axis=1)
        
        temp['Pno'] = [kcom['Pno'].loc[i]]
        temp['Caseno'] = [kcom['Caseno'].loc[i]]
        
        t_idx = 1 
        for period in total:
            #print(period)
            
            if len(period['sym']) > 1:
                for s in period['sym']:
                    temp['Kcom_'+ str(t_idx)] = [s]
                    temp['Kcom_du'+ str(t_idx)] = period['time']
                    t_idx += 1
            else:
                temp['Kcom_'+ str(t_idx)] = period['sym']
                temp['Kcom_du'+ str(t_idx)] = period['time']
                t_idx += 1
        
        kcom_out = kcom_out.append(temp)
        '''
        
        print('%d. %s' % (i+1,kcom['Kcom'].loc[i]))
        #the_file.write('%d. %s\n' % (i+1,kcom['Kcom'].loc[i]))
        for period in total:
            print('時間： %s' % period['time'])
            print('症狀： %s' % ','.join(period['sym']))
            #the_file.write('時間： %s\n' % period['time'])
            #the_file.write('症狀： %s\n' % ','.join(period['sym']))
        print('\n')
        #the_file.write('\n')
    #print(max_length)
    #kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")
    

1. 今天下午開始雙腳抽筋 現求治
時間： 今天下午
症狀： 雙腳抽筋


2. 剛剛在家全身不適 臉潮紅 在家量血壓高 現求治
時間： 剛剛
症狀： 全身不適,臉潮紅,血壓高


3. 據家屬訴今天早上開始食慾差 不說話 全身無力 現意識不清 119測血糖33MG/DL
時間： 今天早上
症狀： 食慾差,不說話,全身無力
時間： 現在
症狀： 意識不清


4. 昨天開始頭暈不適 現未改善
時間： 昨天
症狀： 頭暈,不適


5. 昨天開始左腰 左腹痛 想吐 外院檢查有結石 現持續中
時間： 昨天
症狀： 左腹痛,想吐


6. 腹痛兩天 現求治
時間： 兩天
症狀： 腹痛


7. 昨天晚上開始腹痛 想吐 LMP：8/5
時間： 昨天晚上
症狀： 腹痛,想吐


8. 剛剛睡覺忽然開始頭痛不適
時間： 剛剛
症狀： 頭痛,不適


9. 昨天晚上開始腹痛不適 現求治
時間： 昨天晚上
症狀： 腹痛,不適


10. 今天早上開始發燒 喉嚨痛 咳嗽 肌肉痠痛 外院求治過 現未改善
時間： 今天早上
症狀： 發燒,喉嚨痛,咳嗽,肌肉痠痛


11. 前天下午開始雙手有紅疹現求治
時間： 前天下午
症狀： 雙手紅疹


12. 剛開始發燒 咳嗽多日 現求治
時間： 剛開始
症狀： 發燒
時間： 多日
症狀： 咳嗽


13. 剛開始腹痛  冒冷汗 想吐 現求治
時間： 剛開始
症狀： 腹痛,冒冷汗,想吐


14. 今天下午開始上吐下瀉 現求治
時間： 今天下午
症狀： 上吐下瀉


15. 晚上開始頭暈 吐 有來看過急診 現仍覺頭暈
時間： 晚上
症狀： 頭暈,吐
時間： 現在
症狀： 頭暈


16. 剛開始暈眩 想吐 會喘  現求治
時間： 剛開始
症狀： 想吐,喘


17. 剛剛起床後開始頭暈 現求治
時間： 剛剛
症狀： 頭暈


18. 剛開始頭暈 現求治
時間： 剛開始
症狀： 頭暈


19. 昨天開始發燒不適 現求治
時間： 昨天
症狀： 發燒,不適


20. 8/15號開始腹瀉 腹痛  LMD求治未改善 已停經
時間： 
症狀： 腹瀉,腹痛


21. 今早開始解尿痛 頻尿 現未改善
時間： 今早
症狀： 解尿痛,頻尿


22. 門診抽血肝指數高 頭暈 建議轉急診待床
時間： 
症狀： 頭暈


23. 剛剛開始全身紅疹 現求治
時間： 剛

症狀： 胸口不適


330. 呼吸胸口不適多天 現未改善 SPO2:99%
時間： 多天
症狀： 胸口不適


331. 今天下午三點腹痛 現未改善 LMP:8月初
時間： 今天下午
症狀： 腹痛


332. 今天開始腹痛 LMP:8/25
時間： 今天
症狀： 腹痛


333. 剛開始腹痛  吐現求治
時間： 剛開始
症狀： 腹痛,吐


334. 前天開始發燒 上吐下瀉 月經第二天
時間： 前天
症狀： 發燒,上吐下瀉


335. 發燒 咳嗽多天 現未改善 今天從香港返台
時間： 
症狀： 發燒
時間： 多天
症狀： 咳嗽


336. 發燒 咳嗽多天 現未改善 昨天從香港返台 急診求治過 現未改善
時間： 
症狀： 發燒
時間： 多天
症狀： 咳嗽


337. 早上開始發燒 咳嗽
時間： 早上
症狀： 發燒,咳嗽


338. 腹痛 腹瀉 嘔吐多天 現求治
時間： 
症狀： 腹痛,腹瀉
時間： 多天
症狀： 嘔吐


339. 剛8點多開始頭暈  現求治
時間： 剛
症狀： 頭暈


340. 前天開始腹痛 腹瀉 現求治
時間： 前天
症狀： 腹痛,腹瀉


341. 今天下午開始發燒 現求治
時間： 今天下午
症狀： 發燒


342. 3天前開始  眩暈  頭暈
時間： 3天前
症狀： 眩暈,頭暈


343. 早上開始右腰痛 現求治
時間： 早上
症狀： 右腰痛


344. 今天晚上開始頻尿解尿疼痛 現求治
時間： 今天晚上
症狀： 頻尿,疼痛


345. 今天開始血糖高 想吐 嗜睡
時間： 今天
症狀： 血糖高,想吐,嗜睡


346. 晚上開始覺頭暈不適
時間： 晚上
症狀： 頭暈,不適


347. 昨天開始腹痛 想吐 腹瀉  發燒 現求治 LMP:月經第三天
時間： 昨天
症狀： 腹痛,想吐,腹瀉,發燒


348. 今天下午開始腹痛 畏寒 嘔吐
時間： 今天下午
症狀： 腹痛,畏寒,嘔吐


349. 昨天晚上開始血壓高 現頭痛求治
時間： 昨天晚上
症狀： 血壓高
時間： 現在
症狀： 頭痛


350. 昨天開始情緒激動   有喝酒
時間： 昨天
症狀： 情緒激動


351. 家屬訴 今天安養院發現胃造簍管反抽有咖啡色液


352. 今天下午開始想吐 左腰痛 血尿 現求治
時間： 今天下午
症狀： 想吐,左腰痛,血尿




538. 今早開始頭痛 自行買藥吃未改善 全身骨頭痠痛 現發燒求治
時間： 今早
症狀： 頭痛,痠痛
時間： 現在
症狀： 發燒


539. 剛剛開始解血尿 解尿痛 現求治
時間： 剛剛
症狀： 血尿,解尿痛


540. 昨天開始發燒  腹痛 懷孕8週
時間： 昨天
症狀： 發燒,腹痛


541. 多天前開始解黑便 頭暈 外院求治過 現未改善
時間： 多天前
症狀： 解黑便,頭暈


542. 昨天開始腹痛 上吐下瀉 現求治 LMP：8/17
時間： 昨天
症狀： 腹痛,上吐下瀉


543. 今天晚上開始雙手雙腳會抖頭暈不適
時間： 今天晚上
症狀： 頭暈,不適


544. 剛開始頭暈 嘔吐 現求治 

時間： 剛開始
症狀： 頭暈,嘔吐


545. 剛剛開始胃痛 想吐頭痛  現求治lmp7月底 有服用一顆安眠藥
時間： 剛剛
症狀： 胃痛,想吐,頭痛


546. 家屬訴  昨天開始血壓高 吸吸喘  SPO2 98% 119車上驗血糖--449mg /dl
時間： 昨天
症狀： 血壓高,喘


547. 剛剛上班中吃完口服藥後呼吸喘 雙手抖 現求治 SPO2 100
時間： 剛剛
症狀： 呼吸喘


548. 今天開始腹痛 現求治
時間： 今天
症狀： 腹痛


549. 今天晚上七點開始上腹及胸口疼痛 現求治
時間： 今天晚上
症狀： 胸口疼痛


550. 發燒3天 全身無力 肌肉痠痛 腹瀉 腹痛 腹脹 吃不下 現求治
時間： 3天
症狀： 發燒,全身無力,肌肉痠痛,腹瀉,腹痛,腹脹,吃不下


551. 昨天開始腹瀉 想吐 頭暈  現求治 LMP:7/24
時間： 昨天
症狀： 腹瀉,想吐,頭暈


552. 據自殺防治中心人員代訴剛在中心門口突然情緒激動 躁動 無法控制
時間： 
症狀： 自殺
時間： 剛
症狀： 情緒激動


553. 全身無力多天 外院求治未改善 現求善
時間： 多天
症狀： 全身無力


554. 多天前開始咳嗽有痰 呼吸喘 幻聽 現求治 SPO2 96 外ON O2
時間： 多天前
症狀： 咳嗽,痰,呼吸喘,幻聽


555. 今早開始右腰痛 想吐 現未改善腰痛
時間： 今早
症狀： 右腰痛,想吐
時間： 現在
症狀： 腰痛


556. 今天晚上開始胸悶 全身無力 現求治
時間： 今天晚上
症狀： 胸悶,全身無力




750. 今天洗腎後開始呼吸喘 SPO2:97%
時間： 今天
症狀： 呼吸喘


751. 昨天晚上九點開始腹痛 上吐下瀉 現求治
時間： 昨天晚上
症狀： 腹痛,上吐下瀉


752. 昨天開始頻尿  左腰痛  現求治  LMP:8/7
時間： 昨天
症狀： 頻尿,左腰痛


753. 剛在家頭暈 有吐3次  現血壓高
時間： 剛
症狀： 頭暈,吐
時間： 現在
症狀： 血壓高


754. 今晚9點多開始喘厲害 張口呼吸  外on foley O2
時間： 今晚
症狀： 喘


755. 昨天開始嘔吐 現未改善 已停經
時間： 昨天
症狀： 嘔吐


756. 今天晚上開始NG灌食後病人就會嘔吐 外on NG 7/31(s)
時間： 今天晚上
症狀： 嘔吐


757. 今天下午開始呼吸喘 冒冷汗現求治 外ON NG O2 SPO2 99% 119驗血糖236
時間： 今天下午
症狀： 呼吸喘,冒冷汗


758. 昨天洗完腎開始全身不適 現發燒
時間： 昨天
症狀： 全身不適
時間： 現在
症狀： 發燒


759. 剛剛開始發燒
時間： 剛剛
症狀： 發燒


760. 剛開始腹痛 嘔吐 現未改善
時間： 剛開始
症狀： 腹痛,嘔吐


761. 清晨兩點在家兩血壓高 現頭暈不適
時間： 清晨
症狀： 血壓高
時間： 現在
症狀： 頭暈,不適


762. 剛剛在家量血壓高頭會暈 現求治
時間： 剛剛
症狀： 血壓高,暈


763. 剛開始左膝痛求治
時間： 剛開始
症狀： 左膝痛


764. 喘多天 現發燒求治 SPO2-80%
時間： 多天
症狀： 喘
時間： 現在
症狀： 發燒


765. 腹痛多天 OPD求治過 現未改善 已停經
時間： 多天
症狀： 腹痛


766. 早上開始頭暈 嘔吐
時間： 早上
症狀： 頭暈,嘔吐


767. 三天未解便現腹痛求治
時間： 現在
症狀： 腹痛


768. 早上開始 腹痛 發燒
時間： 早上
症狀： 腹痛,發燒


769. 昨天開始腹痛  現求治
時間： 昨天
症狀： 腹痛


770. 發燒 左腹痛 頭痛 頻尿多天 現仍未改善
時間： 
症狀： 發燒,左腹痛,頭痛
時間： 多天
症狀： 頻尿


771. 剛剛開始呼吸喘  現求治  SPO2:100%
時間： 剛剛
症狀： 呼吸喘


772.

1143. 今天開始胃痛 解黑便
時間： 今天
症狀： 胃痛,解黑便


1144. 今天開始腹痛 腹瀉 現未改善
時間： 今天
症狀： 腹痛,腹瀉


1145. 昨天開始打嗝
時間： 昨天
症狀： 打嗝


1146. 今天開始全身無力 血壓低 現求治
時間： 今天
症狀： 全身無力,血壓低


1147. 發燒多日 現求治
時間： 多日
症狀： 發燒


1148. 昨晚開始 發燒
時間： 昨晚
症狀： 發燒


1149. 今天早上開始腹痛 腹瀉 現未改善 已停經
時間： 今天早上
症狀： 腹痛,腹瀉


1150. 家屬代訴剛剛量血壓低  全身無力 現求治
時間： 剛剛
症狀： 血壓低,全身無力


1151. 剛開始發燒 呼吸喘血氧84
時間： 剛開始
症狀： 發燒,呼吸喘


1152. 胸悶數日 今早開始胸悶症狀加重 現求治
時間： 數日
症狀： 胸悶
時間： 今早
症狀： 胸悶


1153. 剛開始發燒 呼吸喘 現求治血氧80
時間： 剛開始
症狀： 發燒,呼吸喘


1154. 今天早上開始左腰痛  嘔吐  不適
時間： 今天早上
症狀： 左腰痛,嘔吐,不適


1155. 今天開始頭暈 嘔吐 現發燒不適
時間： 今天
症狀： 頭暈,嘔吐
時間： 現在
症狀： 發燒,不適


1156. 昨天開始嘔吐 現未改善
時間： 昨天
症狀： 嘔吐


1157. 今天開始呼吸喘 頭痛 SPO2:100% 外ON NG(8/12 S)
時間： 今天
症狀： 呼吸喘,頭痛


1158. 今天下午開始呼吸喘 現未改善 SPO2: 100% 外ON NG8/12 on (S)
時間： 今天下午
症狀： 呼吸喘


1159. 今天腹脹 8/12置胃管  今天已經來過急診
時間： 今天
症狀： 腹脹


1160. 打嗝4.5天 未改善 現求治
時間： 5天
症狀： 打嗝


1161. 今天開始呼吸喘 現未改善 SPO2:82%
時間： 今天
症狀： 呼吸喘


1162. 今天早上開始解黑便
時間： 今天早上
症狀： 解黑便


1163. 這幾天全身黃疸   門診建議住院
時間： 這幾天
症狀： 全身黃疸


1164. 家屬代訴剛剛喝酒後約抽筋一分鐘 現意識不清 口吐白沫 直入  (舊號:6169580)
時間： 剛剛
症狀： 抽筋
時間： 現在
症狀

...

5816. 昨天晚上胃痛  今天早上開始頭痛 畏寒 現求治
時間： 昨天晚上
症狀： 胃痛
時間： 今天早上
症狀： 頭痛,畏寒


5817. 今天下午開始胃痛 現仍不適求治
時間： 今天下午
症狀： 胃痛
時間： 現在
症狀： 不適


5818. 頭痛 雙耳耳鳴半年 剛剛開始咳嗽 有血絲 現求治
時間： 
症狀： 頭痛
時間： 半年
症狀： 雙耳耳鳴
時間： 剛剛
症狀： 咳嗽


5819. 剛剛打針後覺得舌頭歪一邊  現求治


5820. 腹痛 晚上開始 想吐
時間： 
症狀： 腹痛
時間： 晚上
症狀： 想吐


5821. 凌晨開始上吐下瀉 現未改善 LMP:12/24
時間： 凌晨
症狀： 上吐下瀉


5822. 今天凌晨四點開始 右手大拇指食指會麻沒法向上舉
時間： 今天凌晨
症狀： 麻


5823. 發燒 咳嗽兩週  現未改善
時間： 
症狀： 發燒
時間： 兩週
症狀： 咳嗽


5824. 今天下午14點開始胸悶 呼吸喘 現求治 SPO2:97%
時間： 今天下午
症狀： 胸悶,呼吸喘


5825. 昨天晚上開始呼吸喘 現仍不適 SPO2：90%
時間： 昨天晚上
症狀： 呼吸喘
時間： 現在
症狀： 不適


5826. 家人代訴今天下午開始發燒 吐 現仍未改善
時間： 今天下午
症狀： 發燒,吐


5827. 剛剛吃東西忽然昏倒 現意識不清 直入
時間： 剛剛
症狀： 昏倒
時間： 現在
症狀： 意識不清


5828. 今早開始嘔吐 頭暈 頭痛 現發燒  LMP:4/23
時間： 今早
症狀： 嘔吐,頭暈,頭痛
時間： 現在
症狀： 發燒


5829. 剛吃完飯後開始噁心 冒冷汗 有喝酒 現血壓低
時間： 剛
症狀： 冒冷汗
時間： 現在
症狀： 血壓低


5830. 週2開始發燒 現仍發燒
時間： 
症狀： 發燒
時間： 現在
症狀： 發燒


5831. 昨天開始全身不適嘔吐黑色嘔吐物 現發燒求治
時間： 昨天
症狀： 全身不適,嘔吐
時間： 現在
症狀： 發燒


5832. 昨天開始腹痛 想吐 現未改善 LMP5/19
時間： 昨天
症狀： 腹痛,想吐


5833. 頭暈數天 有喝酒 現求治
時間： 數天
症狀： 頭暈


5834. 昨天晚上咳嗽 流鼻水 全身無力 現求治
時間： 昨天晚上
症狀： 咳嗽,流鼻水

時間： 昨天
症狀： 腹痛
時間： 現在
症狀： 發燒,不適


6052. 咳嗽多天現仍不適
時間： 多天
症狀： 咳嗽
時間： 現在
症狀： 不適


6053. 下午兩點胸口悶 已含 NTG 兩顆


6054. 剛剛開始呼吸喘 心悸  現求治
時間： 剛剛
症狀： 呼吸喘,心悸


6055. 昨天開始咳嗽 流鼻水 喉嚨痛 全身無力不適 現持續中
時間： 昨天
症狀： 咳嗽,流鼻水,喉嚨痛,全身無力,不適


6056. 剛剛開始胸悶不適雙腳腫  現求治
時間： 剛剛
症狀： 胸悶,不適,雙腳腫


6057. 今天開始喉嚨痛 頭痛 現求治
時間： 今天
症狀： 喉嚨痛,頭痛


6058. 今天早上開始腹痛 現仍不適
時間： 今天早上
症狀： 腹痛
時間： 現在
症狀： 不適


6059. 今天下午5點多腹痛 想腹瀉 在廁所暈倒  LMD求治建議轉診
時間： 今天下午
症狀： 腹痛,腹瀉,暈倒


6060. 星期一開始腹瀉 打噴嚏 吃不下 全身無力 躺不下 現求治 已停經
時間： 星期一
症狀： 腹瀉,打噴嚏,吃不下,全身無力


6061. 今天吃完晚餐後 全身過敏 現求治
時間： 今天
症狀： 全身過敏


6062. 今早始頭暈 想吐 腹脹 現持續中
時間： 今早
症狀： 頭暈,想吐,腹脹


6063. 剛剛開始腹痛 頭暈 腹瀉 LMP：4/18
時間： 剛剛
症狀： 腹痛,頭暈,腹瀉


6064. 早上開始頭暈 解血尿
時間： 早上
症狀： 頭暈,血尿


6065. 發燒4天 喉嚨破 想吐 腹瀉 頭痛 全身無力 LMP:4月中現求治
時間： 4天
症狀： 發燒,想吐,腹瀉,頭痛,全身無力


6066. 今天開始解血尿 腰痠 解尿疼痛 頻尿 現求治
時間： 今天
症狀： 血尿,腰痠,疼痛,頻尿


6067. 中午開始 咳嗽 流鼻水 發燒 喉嚨癢
時間： 中午
症狀： 咳嗽,流鼻水,發燒,喉嚨癢


6068. 要求開藥


6069. 發燒 4天 想吐 盜汗 吃不好 咳嗽 有痰
時間： 
症狀： 發燒,想吐,吃不好,咳嗽,痰


6070. 昨天開始頭暈 頭痛 咳嗽 全身無力 現求治
時間： 昨天
症狀： 頭暈,頭痛,咳嗽,全身無力


6071. 咳嗽咳到胸口痛會喘 五天 今天開始左耳疼痛 SPO2 96%
時間： 
症狀： 咳嗽,胸

6316. 昨晚開始腹瀉4-8次 腹脹 現求治
時間： 昨晚
症狀： 腹瀉,腹脹


6317. 今天開始腹痛 現求治 lmp5月中
時間： 今天
症狀： 腹痛


6318. 多天前開始背痛 腰痠  現求治
時間： 多天前
症狀： 背痛,腰痠


6319. 昨天剛出院 昨天晚上開始 咳嗽有痰 呼吸喘 spo2 92%
時間： 昨天晚上
症狀： 咳嗽,痰,呼吸喘


6320. 今天開始發燒 全身發麻 腹痛 現求治 LMP:4月底
時間： 今天
症狀： 發燒,全身發麻,腹痛


6321. 昨天開始呼吸喘 漏尿 外院求治 建議轉診  SPO2 98
時間： 昨天
症狀： 呼吸喘


6322. 剛剛在家吃飯後 全身無力 四肢發麻 119驗血糖231 現求治
時間： 剛剛
症狀： 全身無力,發麻


6323. 昨天開始胸痛 現求治
時間： 昨天
症狀： 胸痛


6324. 剛剛開始腹痛 上吐下瀉 現求治 LMP:5/2
時間： 剛剛
症狀： 腹痛,上吐下瀉


6325. 今天下午開始頭暈 全身起紅疹 診所求治過 建議轉急診
時間： 今天下午
症狀： 頭暈,全身起紅疹


6326. 今天下午開始頭痛 胃痛 現求治  LMP:4月底
時間： 今天下午
症狀： 頭痛,胃痛


6327. 家屬代訴絕食多天  現頭暈求治
時間： 現在
症狀： 頭暈


6328. 感冒多天 現求治 5//3由福建返台


6329. 頭暈眼花多天 診所求治過 現仍未改善


6330. 剛剛開始胃痛  現求治
時間： 剛剛
症狀： 胃痛


6331. 剛剛吃晚餐後全身起紅疹會癢  現求治
時間： 剛剛
症狀： 全身起紅疹,癢


6332. 外傭待訴今天下午開始呼吸喘  血壓低 現求治 SPO2:95% 外ON Tr O2
時間： 今天下午
症狀： 呼吸喘,血壓低


6333. 今天晚上開始喘 現求治 SPO2 100%
時間： 今天晚上
症狀： 喘


6334. 今天晚上十點半開始右臉抽痛 現求治
時間： 今天晚上
症狀： 痛


6335. 昨天晚上開始上吐下瀉 現求治 LMP:4月底
時間： 昨天晚上
症狀： 上吐下瀉


6336. 全身痠痛要求自費買藥
時間： 
症狀： 全身痠痛


6337. 今天晚上開始全身痠痛 發燒 腹痛 現仍不適 LMP 4/27
時間： 

6615. 呼吸時會胸痛多天 現胸悶未改善
時間： 多天
症狀： 胸痛
時間： 現在
症狀： 胸悶


6616. 流鼻水一星期 今天開始咳嗽
時間： 一
症狀： 流鼻水
時間： 今天
症狀： 咳嗽


6617. 剛剛開始腹痛  腹瀉  現求治 LMP:4/27
時間： 剛剛
症狀： 腹痛,腹瀉


6618. 全身紅疹 癢多天 現求治
時間： 
症狀： 全身紅疹
時間： 多天
症狀： 癢


6619. 昨天晚上開始臉潮紅 心悸不適 現求治
時間： 昨天晚上
症狀： 臉潮紅,心悸,不適


6620. 晚上開始上腹痛 現求治
時間： 晚上
症狀： 腹痛


6621. 昨天開始下腹痛 頭痛 流鼻水 咳嗽  現求治 LMP:4月中
時間： 昨天
症狀： 腹痛,頭痛,流鼻水,咳嗽


6622. 剛開始呼吸喘 臉色發白 SPO2 97%
時間： 剛開始
症狀： 呼吸喘


6623. 今天下午4點多開始腹痛抽痛 血氧97% 現發燒 AP26週
時間： 今天下午
症狀： 腹痛,痛
時間： 現在
症狀： 發燒


6624. 頭痛 流鼻水 咳嗽2天   現求治
時間： 
症狀： 頭痛,流鼻水
時間： 2天
症狀： 咳嗽


6625. 左手疼痛多天 現未改善
時間： 多天
症狀： 左手疼痛


6626. 今天早上10點多開始感到心悸 冒冷汗 頭暈 現仍持續中 懷孕7個多月
時間： 今天早上
症狀： 心悸,冒冷汗,頭暈


6627. 晚上開始左手 左臉麻 頭暈
時間： 晚上
症狀： 左臉麻,頭暈


6628. 剛剛開始胸悶 頭暈 現求治
時間： 剛剛
症狀： 胸悶,頭暈


6629. 左側無力 說話不清3-4天 LMD求治未改善 現求治
時間： 
症狀： 左側無力
時間： 4天
症狀： 說話不清


6630. 今天下午四點開始右手無力 說話不清 現求治
時間： 今天下午
症狀： 右手無力,說話不清


6631. 今天開始左腰痛 腹痛  想吐 現求治 LMP:月經第四天
時間： 今天
症狀： 左腰痛,腹痛,想吐


6632. 多天前 右腳趾紅腫 門診建議轉急診
時間： 多天前
症狀： 右腳趾紅腫


6633. 要求開藥


6634. 昨天晚上開始腹瀉 現求治
時間： 昨天晚上
症狀： 腹瀉


6635. 四天前開始咳嗽有痰  昨天開始發燒  OPD診斷P

KeyboardInterrupt: 

In [ ]:
kcom_out.to_csv('kcom_output_0526.csv', index=False,encoding="utf_8_sig")
#kcom_out.head()

In [ ]:
#import time
print("--- %s seconds ---" % (timer.time() - start_time))

In [ ]:
#kcom_out.to_csv('kcom_output.csv', index=False,encoding="utf_8_sig")